In [14]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import random 

In [15]:
# Собственный кейс c экспериментальными данными

# В таблице user-actions сохраняются действия покупателей в интернет магазине, структура таблицы:

# user_id - id пользователя 
# product_id - id товара 
# action - действие, просмотр товара, сохранение в корзину, покупка 
# time - время совершения события 
# date - дата события 

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой?
# Какой percantage добавления в корзину по отношению к покупкам? 

In [16]:
## Подготовка эксперементальных данных (можно пропустить прямо до SQL запроса)

all_users_ids = np.arange(1, 1000)
all_products_ids = np.arange(1, 100)
n = 10000 

In [17]:
# Выбираем рандомом 10000 раз юзер id из диапозона от 1 до 1000 и продуктовые id из диапозона от 1 до 100
user_ids = np.random.choice(all_users_ids, n)
product_ids = np.random.choice(all_products_ids, n)

In [18]:
# Дата отсчета и 10000 значений от этой даты с разницей в 5 минут каждая 
start_date = pd.to_datetime('2022-03-18')

times = pd.date_range(start=start_date, periods=n, freq='5min')

In [19]:
# создание df и последующее добавление к нему факта просмотра продукта: "view"
user_actions = pd.DataFrame({
    'user_id': user_ids,
    'product_id': product_ids,
    'time': times,
})

user_actions['action'] = 'view'

In [20]:
def generate_funel_actions(user_id, product_id, time):
    """Формирование на основе случайных данных датасета с различными action,
    учитывая специфику данных (вероятности добавления в корзину, покупки,
    а также изменение времени в зависимости от action)
    view -> add_to_cart -> to_purchase,
    где view - просмотр продукта,
    add_to_cart - добавление в корзину продукта 
    и to_purchase - факт покупки товара"""
    
    add_to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    if np.random.binomial(1, add_to_cart, 1)[0]:
        df = pd.DataFrame({
                            'user_id': user_id,
                            'product_id': product_id,
                            'time': time + pd.Timedelta(7.5, unit='s'),
                            'action': 'add_to_cart'}, index=[0])
        if np.random.binomial(1, to_purchase, 1)[0]: 
            df_purchase = pd.DataFrame({
                            'user_id': user_id,
                            'product_id': product_id,
                            'time': time + pd.Timedelta(15, unit='s'),
                            'action': 'purchase'}, index=[0])  
            
            df = df.append(df_purchase)
            
    return df

In [21]:
# Добляем с помощью функции изменения во временный df
to_cart_df = pd.DataFrame()

for index, row in user_actions.iterrows(): 
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)

to_cart_df.head()

,user_id,product_id,time,action
0,853,58,2022-03-18 00:20:07.500,add_to_cart
0,272,62,2022-03-18 00:45:07.500,add_to_cart
0,272,62,2022-03-18 00:45:15.000,purchase
0,567,96,2022-03-18 00:55:07.500,add_to_cart
0,567,96,2022-03-18 00:55:15.000,purchase


In [22]:
# изменяем основной df и сортируем по времени операции
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

user_actions['date'] = user_actions.time.dt.date

user_actions.head()

,user_id,product_id,time,action,date
0,717,63,2022-03-18 00:00:00,view,2022-03-18
1,705,7,2022-03-18 00:05:00,view,2022-03-18
2,863,75,2022-03-18 00:10:00,view,2022-03-18
3,698,21,2022-03-18 00:15:00,view,2022-03-18
4,853,58,2022-03-18 00:20:00,view,2022-03-18


In [23]:
# Для каждого дня рассчитайте конверсию, какой процент просмотров|добавления в корзину товаров завершился покупкой?

query_1 = """SELECT date,
                    views, 
                    carts,
                    purchases,
                    100 * purchases / views AS purchase_percantage_by_views,
                    100 * purchases / carts AS purchase_percantage_by_carts
             FROM (
                 SELECT date,
                        COUNT(CASE WHEN action = 'view'        THEN 1 ELSE NULL END) AS views,
                        COUNT(CASE WHEN action = 'add_to_cart' THEN 1 ELSE NULL END) AS carts,
                        COUNT(CASE WHEN action = 'purchase'    THEN 1 ELSE NULL END) AS purchases
             FROM user_actions
             GROUP BY date);"""

sqldf(query_1)

,date,views,carts,purchases,purchase_percantage_by_views,purchase_percantage_by_carts
0,2022-03-18,288,56,25,8,44
1,2022-03-19,288,50,21,7,42
2,2022-03-20,288,70,27,9,38
3,2022-03-21,288,49,16,5,32
4,2022-03-22,288,55,20,6,36
5,2022-03-23,288,61,22,7,36
6,2022-03-24,288,65,32,11,49
7,2022-03-25,288,61,17,5,27
8,2022-03-26,288,67,31,10,46
9,2022-03-27,288,58,25,8,43
